# การปรับแต่งโมเดล Open AI

สมุดบันทึกนี้อ้างอิงจากคำแนะนำปัจจุบันที่ให้ไว้ในเอกสาร [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) จาก Open AI

การปรับแต่งช่วยปรับปรุงประสิทธิภาพของโมเดลพื้นฐานสำหรับแอปพลิเคชันของคุณโดยการฝึกซ้ำด้วยข้อมูลและบริบทเพิ่มเติมที่เกี่ยวข้องกับกรณีการใช้งานหรือสถานการณ์เฉพาะนั้น โปรดทราบว่าเทคนิคการออกแบบพรอมต์เช่น _few shot learning_ และ _retrieval augmented generation_ ช่วยให้คุณเพิ่มประสิทธิภาพพรอมต์เริ่มต้นด้วยข้อมูลที่เกี่ยวข้องเพื่อปรับปรุงคุณภาพ อย่างไรก็ตาม วิธีการเหล่านี้ถูกจำกัดด้วยขนาดหน้าต่างโทเค็นสูงสุดของโมเดลพื้นฐานเป้าหมาย

ด้วยการปรับแต่ง เรากำลังฝึกซ้ำโมเดลเองด้วยข้อมูลที่ต้องการ (ช่วยให้เราใช้ตัวอย่างได้มากกว่าที่จะใส่ในหน้าต่างโทเค็นสูงสุดได้) - และปรับใช้เวอร์ชัน _กำหนดเอง_ ของโมเดลที่ไม่จำเป็นต้องมีตัวอย่างในเวลาทำนายผล ซึ่งไม่เพียงแต่ช่วยปรับปรุงประสิทธิผลของการออกแบบพรอมต์ของเรา (เรามีความยืดหยุ่นมากขึ้นในการใช้หน้าต่างโทเค็นสำหรับสิ่งอื่น ๆ) แต่ยังอาจช่วยลดต้นทุนของเรา (โดยลดจำนวนโทเค็นที่ต้องส่งไปยังโมเดลในเวลาทำนายผล)

การปรับแต่งมี 4 ขั้นตอน:
1. เตรียมข้อมูลการฝึกและอัปโหลด
1. รันงานฝึกเพื่อรับโมเดลที่ปรับแต่งแล้ว
1. ประเมินโมเดลที่ปรับแต่งและปรับปรุงคุณภาพ
1. ปรับใช้โมเดลที่ปรับแต่งสำหรับการทำนายผลเมื่อพอใจแล้ว

โปรดทราบว่าไม่ใช่ทุกโมเดลพื้นฐานที่รองรับการปรับแต่ง - [ตรวจสอบเอกสาร OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) สำหรับข้อมูลล่าสุด คุณยังสามารถปรับแต่งโมเดลที่เคยถูกปรับแต่งมาก่อนได้ ในบทช่วยสอนนี้ เราจะใช้ `gpt-35-turbo` เป็นโมเดลพื้นฐานเป้าหมายสำหรับการปรับแต่ง

---


### ขั้นตอนที่ 1.1: เตรียมชุดข้อมูลของคุณ

มาสร้างแชทบอทที่ช่วยให้คุณเข้าใจตารางธาตุโดยการตอบคำถามเกี่ยวกับธาตุด้วยบทกลอนลิมเมอริก ใน _บทเรียนง่ายๆ_ นี้ เราจะสร้างชุดข้อมูลเพื่อฝึกโมเดลด้วยตัวอย่างการตอบไม่กี่ตัวอย่างที่แสดงรูปแบบข้อมูลที่คาดหวัง ในกรณีใช้งานจริง คุณจะต้องสร้างชุดข้อมูลที่มีตัวอย่างมากกว่านี้มาก คุณอาจจะสามารถใช้ชุดข้อมูลเปิด (สำหรับโดเมนแอปพลิเคชันของคุณ) หากมี และปรับรูปแบบเพื่อใช้ในการปรับแต่งโมเดล

เนื่องจากเรากำลังเน้นที่ `gpt-35-turbo` และมองหาการตอบแบบรอบเดียว (chat completion) เราสามารถสร้างตัวอย่างโดยใช้ [รูปแบบที่แนะนำนี้](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) ซึ่งสะท้อนความต้องการของ OpenAI chat completion หากคุณคาดหวังเนื้อหาการสนทนาแบบหลายรอบ คุณจะใช้ [รูปแบบตัวอย่างหลายรอบ](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) ซึ่งรวมพารามิเตอร์ `weight` เพื่อระบุข้อความที่ควรใช้ (หรือไม่ใช้) ในกระบวนการปรับแต่ง

เราจะใช้รูปแบบรอบเดียวที่ง่ายกว่าสำหรับบทเรียนนี้ ข้อมูลอยู่ในรูปแบบ [jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) โดยมี 1 ระเบียนต่อบรรทัด แต่ละระเบียนแสดงเป็นอ็อบเจ็กต์ JSON รูปแบบ ตัวอย่างด้านล่างแสดง 2 ระเบียนเป็นตัวอย่าง - ดู [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) สำหรับชุดตัวอย่างเต็ม (10 ตัวอย่าง) ที่เราจะใช้สำหรับบทเรียนการปรับแต่งโมเดล **หมายเหตุ:** แต่ละระเบียน _ต้อง_ ถูกกำหนดในบรรทัดเดียว (ไม่แบ่งบรรทัดเหมือนในไฟล์ JSON ที่จัดรูปแบบ)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

ในกรณีใช้งานจริง คุณจะต้องมีชุดตัวอย่างที่ใหญ่กว่านี้มากเพื่อผลลัพธ์ที่ดี - การแลกเปลี่ยนจะอยู่ระหว่างคุณภาพของคำตอบกับเวลา/ค่าใช้จ่ายในการปรับแต่ง เราใช้ชุดเล็กเพื่อให้สามารถปรับแต่งได้อย่างรวดเร็วเพื่อแสดงกระบวนการ ดู [ตัวอย่าง OpenAI Cookbook นี้](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) สำหรับบทเรียนการปรับแต่งที่ซับซ้อนมากขึ้น


---

### ขั้นตอนที่ 1.2 อัปโหลดชุดข้อมูลของคุณ

อัปโหลดข้อมูลโดยใช้ Files API [ตามที่อธิบายไว้ที่นี่](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file) โปรดทราบว่าเพื่อให้รันโค้ดนี้ได้ คุณต้องทำตามขั้นตอนต่อไปนี้ก่อน:
 - ติดตั้งแพ็กเกจ `openai` สำหรับ Python (ตรวจสอบให้แน่ใจว่าใช้เวอร์ชัน >=0.28.0 เพื่อฟีเจอร์ล่าสุด)
 - ตั้งค่าตัวแปรสภาพแวดล้อม `OPENAI_API_KEY` เป็นคีย์ API ของ OpenAI ของคุณ
หากต้องการเรียนรู้เพิ่มเติม ดู [คู่มือการตั้งค่า](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) ที่จัดเตรียมไว้สำหรับหลักสูตรนี้

ตอนนี้ ให้รันโค้ดเพื่อสร้างไฟล์สำหรับอัปโหลดจากไฟล์ JSONL ในเครื่องของคุณ


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### ขั้นตอนที่ 2.1: สร้างงาน Fine-tuning ด้วย SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### ขั้นตอนที่ 2.2: ตรวจสอบสถานะของงาน

นี่คือสิ่งที่คุณสามารถทำได้กับ API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - แสดงรายการงาน fine-tuning ล่าสุด n งาน
- `client.fine_tuning.jobs.retrieve(<job_id>)` - รับรายละเอียดของงาน fine-tuning เฉพาะงาน
- `client.fine_tuning.jobs.cancel(<job_id>)` - ยกเลิกงาน fine-tuning
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - แสดงรายการเหตุการณ์สูงสุด n เหตุการณ์จากงาน
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

ขั้นตอนแรกของกระบวนการคือ _การตรวจสอบความถูกต้องของไฟล์ฝึกอบรม_ เพื่อให้แน่ใจว่าข้อมูลอยู่ในรูปแบบที่ถูกต้องแล้ว


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### ขั้นตอนที่ 2.3: ติดตามเหตุการณ์เพื่อตรวจสอบความก้าวหน้า


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### ขั้นตอนที่ 2.4: ดูสถานะในแดชบอร์ด OpenAI


คุณยังสามารถดูสถานะได้โดยการเยี่ยมชมเว็บไซต์ OpenAI และสำรวจส่วน _Fine-tuning_ ของแพลตฟอร์ม ซึ่งจะแสดงสถานะของงานปัจจุบัน และยังช่วยให้คุณติดตามประวัติการรันงานก่อนหน้าได้ ในภาพหน้าจอนี้ คุณจะเห็นว่างานก่อนหน้าล้มเหลว และการรันครั้งที่สองสำเร็จ สำหรับบริบท เหตุการณ์นี้เกิดขึ้นเมื่อการรันครั้งแรกใช้ไฟล์ JSON ที่มีบันทึกที่จัดรูปแบบไม่ถูกต้อง - เมื่อแก้ไขแล้ว การรันครั้งที่สองก็เสร็จสมบูรณ์และทำให้โมเดลพร้อมใช้งานได้

![Fine-tuning job status](../../../../../translated_images/th/fine-tuned-model-status.563271727bf7bfba.png)


คุณยังสามารถดูข้อความสถานะและเมตริกได้โดยเลื่อนลงไปด้านล่างเพิ่มเติมในแดชบอร์ดภาพตามที่แสดง:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/th/fine-tuned-messages-panel.4ed0c2da5ea1313b.png) |  ![Metrics](../../../../../translated_images/th/fine-tuned-metrics-panel.700d7e4995a65229.png)|


---

### ขั้นตอนที่ 3.1: ดึง ID & ทดสอบโมเดลที่ปรับแต่งแล้วในโค้ด


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### ขั้นตอนที่ 3.2: โหลดและทดสอบโมเดลที่ปรับแต่งแล้วใน Playground

ตอนนี้คุณสามารถทดสอบโมเดลที่ปรับแต่งแล้วได้สองวิธี วิธีแรกคือคุณสามารถเยี่ยมชม Playground และใช้เมนู Models แบบเลื่อนลงเพื่อเลือกโมเดลที่คุณเพิ่งปรับแต่งจากตัวเลือกที่แสดง วิธีที่สองคือใช้ตัวเลือก "Playground" ที่แสดงในแผง Fine-tuning (ดูภาพหน้าจอด้านบน) ซึ่งจะเปิดมุมมอง _เปรียบเทียบ_ ดังต่อไปนี้ที่แสดงเวอร์ชันโมเดลพื้นฐานและโมเดลที่ปรับแต่งแล้วเคียงข้างกันเพื่อการประเมินอย่างรวดเร็ว

![Fine-tuning job status](../../../../../translated_images/th/fine-tuned-playground-compare.56e06f0ad8922016.png)

เพียงกรอกบริบทของระบบที่ใช้ในข้อมูลการฝึกของคุณและระบุคำถามทดสอบของคุณ คุณจะสังเกตเห็นว่าทั้งสองฝั่งจะได้รับการอัปเดตด้วยบริบทและคำถามที่เหมือนกัน รันการเปรียบเทียบและคุณจะเห็นความแตกต่างของผลลัพธ์ระหว่างกัน _สังเกตว่าโมเดลที่ปรับแต่งแล้วจะแสดงคำตอบในรูปแบบที่คุณให้ไว้ในตัวอย่างของคุณ ในขณะที่โมเดลพื้นฐานจะทำตามคำสั่งของระบบอย่างง่ายๆ_

![Fine-tuning job status](../../../../../translated_images/th/fine-tuned-playground-launch.5a26495c983c6350.png)

คุณจะสังเกตเห็นว่าการเปรียบเทียบนี้ยังแสดงจำนวนโทเค็นสำหรับแต่ละโมเดล และเวลาที่ใช้ในการอนุมาน **ตัวอย่างเฉพาะนี้เป็นตัวอย่างง่ายๆ ที่มีไว้เพื่อแสดงกระบวนการเท่านั้น ไม่ได้สะท้อนชุดข้อมูลหรือสถานการณ์จริง** คุณอาจสังเกตเห็นว่าตัวอย่างทั้งสองแสดงจำนวนโทเค็นเท่ากัน (บริบทของระบบและคำสั่งผู้ใช้เหมือนกัน) โดยโมเดลที่ปรับแต่งแล้วใช้เวลามากกว่าในการอนุมาน (โมเดลที่กำหนดเอง)

ในสถานการณ์จริง คุณจะไม่ใช้ตัวอย่างเล่นๆ แบบนี้ แต่จะปรับแต่งกับข้อมูลจริง (เช่น แคตตาล็อกสินค้าเพื่อบริการลูกค้า) ซึ่งคุณภาพของคำตอบจะเห็นได้ชัดเจนมากขึ้น ใน _บริบทนั้น_ การได้คุณภาพคำตอบที่เทียบเท่ากับโมเดลพื้นฐานจะต้องใช้การออกแบบคำสั่งที่กำหนดเองมากขึ้น ซึ่งจะเพิ่มการใช้โทเค็นและอาจเพิ่มเวลาการประมวลผลสำหรับการอนุมาน _หากต้องการลองดู ให้ตรวจสอบตัวอย่างการปรับแต่งใน OpenAI Cookbook เพื่อเริ่มต้น_

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษาอัตโนมัติ [Co-op Translator](https://github.com/Azure/co-op-translator) แม้เราจะพยายามให้ความถูกต้องสูงสุด แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาต้นทางถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลโดยผู้เชี่ยวชาญมนุษย์ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดใด ๆ ที่เกิดจากการใช้การแปลนี้
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
